In [ ]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

In [ ]:
def sacar_tuplas_oscars(archivo):
    """
    Extrae las tuplas de datos de una archivo CSV.

    Parameters:
    archivo (str): La ruta del archivo CSV.

    Returns:
    list: Una lista de tuplas que contienen los datos extraídos del archivo CSV.
    """

    # Lee el archivo CSV y carga los datos en un DataFrame
    df_detalles = pd.read_csv(archivo)
    
    # Lista para almacenar las tuplas extraídas
    val = []

    # Recorre cada fila del DataFrame
    for pelicula in df_detalles.values:
        
        # Convierte la fila en una lista
        pelicula = list(pelicula)
        
        # Itera sobre cada elemento de la lista
        for i, elemento in enumerate(pelicula):
            
            # Reemplaza los valores NaN con None
            if elemento is np.nan:
                pelicula[i] = None
            
            # Intenta convertir los elementos de tipo str a float o int si es posible
            if isinstance(elemento, str):
                if "." in elemento:
                    try:
                        pelicula[i] = float(elemento)
                    except:
                        pass
                
                if any(caracter.isdigit() for caracter in elemento):
                    try:
                        pelicula[i] = int(elemento)
                    except:
                        pass
                    
                if elemento.startswith('-') and elemento.endswith('-'):
                    try:
                        pelicula[i] = None
                    except:
                        pass
            
        val.append(tuple(pelicula))
        
    # Retorna la lista de tuplas
    return val

In [ ]:
# Asigna a la variable el archivo csv que queremos importar a la BBDD

archivo = ""

In [ ]:
# Establece la conexión con la el servidor MySQL y la BBDD

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')


# Inserta los datos usando la llamada a la función en la tabla deseada

mycursor = cnx.cursor()
query = "INSERT INTO PremiosOscar (Anio, MejorPelicula, MejorDirector, MejorActor, MejorActriz, IdPelicula) VALUES (%s, %s, %s, %s, %s, %s)"
val = sacar_tuplas_oscars(archivo)

try: 
    mycursor.executemany(query, val)
    cnx.commit()
    print(mycursor.rowcount, "registro insertado.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)